# Classy: Data Insights Take Home Exam
## Author: Aaron Trefler
## Fall 2017 Data Science Internship Candidate

# Setup

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.offline as offline
import plotly.plotly as py
import seaborn as sns

from scipy.stats import bernoulli, expon, lognorm, multinomial, randint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# Script Variables
N = 50000 # data sample size

In [ ]:
# Plotting Setup
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
offline.init_notebook_mode()

# Access a Data Source

US state population data taken from the official US Census website. Data estimation accurate as of 7/1/2016.  

- Source: https://www2.census.gov/programs-surveys/popest/datasets/2010-2016/state/asrh/  
- Filename: scprc-est2016-18+pop-res.csv  
- Dataset Description: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2016/SCPRC-EST2016-18+POP-RES.pdf
- Data Estimation Methodology: https://www2.census.gov/programs-surveys/popest/technical-documentation/methodology/2010-2016/2016-natstcopr-meth.pdf 

In [ ]:
# load in data
data_pop = 'scprc-est2016-18+pop-res.csv'
df_pop = pd.read_csv('../data/' + data_pop)

# extract relevant columns
df_pop_state = df_pop[['NAME', 'POPESTIMATE2016']]

# remove non-state entries
df_pop_state = df_pop_state[df_pop_state.NAME != 'United States']
df_pop_state = df_pop_state[df_pop_state.NAME != 'District of Columbia']
df_pop_state = df_pop_state[df_pop_state.NAME != 'Puerto Rico Commonwealth']

In [ ]:
print("Data Verification")
print("\nFirst 5 Entries:")
print(df_pop_state.head())
print("\nLast 5 Entries:")
print(df_pop_state.tail())
print("\nNumber of States:", df_pop_state.shape[0])
print("\nNaN or Missing Values")
print(np.sum(df_pop_state.isnull()))

# Generate Raw Data

## Age 

Modeling approximations, estimates, and assumptions:
- Age is modeled as a uniform probability distribution from 0 to 54. Approximation based on US 2010 Census data (see Figure 1 below). 
- Age is modeled as an exponential probability distribution (lambda = 0.05) for ages greater than 54. Approximation based on US 2010 Census data (see Figure 1 below). 
- No ages greater than 113 were generated. If a value greater than 113 was generated, the value was discarded and re-sampled. Assumption based on an article published in the New York Post that stated that as of 02/09/17 the oldest person in America was 113.  
- 72% of the population is estimated to be 54 or younger. Estimation taken from 2015 data provided by the Henry J. Kaiser Family Foundation.
- Young ages (i.e., 0-18) were allowed to be generated, as donations could be made in the names of these children but the money supplied by their parents, god parents, other family members, etc.


Sources:
- 2010 US Census, "Age and Sex Composition: 2010": https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf
- New York Post Article, "The oldest person in America has died": http://nypost.com/2017/02/09/the-oldest-person-in-america-has-died/ 
- The Henry J. Kaiser Family Foundation, "Population Distribution by Age": http://www.kff.org/other/state-indicator/distribution-by-age/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D   

Note: 
- Continuous values sampled from the exponential distribution were converted to be discrete.

### Figure 1
![US Census Age Histogram](../resources/USCensus_AgeHistogram.png)

In [ ]:
age_data = np.empty(N)
max_age = 113

# bernoulli parameter
p = .72

for i in range(N):
    # generate bernoulli outcome (1="54 and under", 0="over 54")
    x = bernoulli.rvs(p, size = 1)
    
    # generate appropriate age based on bernoulli outcome
    if (x == 1):
        # model "54 and under" as a discrete uniform distribution
        min_value = 0
        max_value = 54
        age_data[i] = randint.rvs(low=min_value, high=max_value+1, size=1)
    else:
        # model "over 54" as an exponential distribution
        decay_rate = .05;
        min_value = 55
        age_data[i] = np.floor(expon.rvs(loc=min_value, scale=1.0/decay_rate, size=1).astype(int))
        # re-sample if age drawn is greater than max_age
        while (age_data[i] > max_age):
            age_data[i] = np.floor(expon.rvs(loc=min_value, scale=1.0/decay_rate, size=1).astype(int))

## Gender
Modeling approximations, estimates, and assumptions:
- 50% of the population is estimated to be male, and the other 50% to be female. Approximation supported by 2010 US Census data, which stated that as of 2010 50.8% of the population was female and the rest were male.

Sources:
- 2010 US Census, "Age and Sex Composition: 2010": https://www.census.gov/prod/cen2010/briefs/c2010br-03.pdf

In [ ]:
p = .5
gender_data = bernoulli.rvs(p, size = N)

## Gift Size

Modeling approximations, estimates, and assumptions:
- Assuming that "Gift Size" refers to single event online donations (i.e., non "monthly" or otherwise automatically recurring donations).
- Average gift size is estimated to be $125. Estimate taken from a Classy.org blog post, "Choosing the Right Giving Levels for Your Online Donation Form".
- Gift size is modeled as a log-normal probability distribution. A log-normal distribution was chosen for the following reasons: (1) it does not generate negative numbers, as all gifts are  non-negative in value; (2) the majority of generated numbers fall close to the mean, as would be expected of most gifts; and (3) it has a long right tail that allows for generated values to be far greater than the mean, as there exists some wealthy patrons who can afford to make very large donations.

Sources:
- Classy blog post, "Choosing the Right Giving Levels for Your Online Donation Form": https://www.classy.org/blog/choosing-the-right-giving-levels-for-your-online-donation-form/

Note:
- 1 cent was added to all generated Gift Size values in order to avoid generating Gift Sizes values of zero dollars.

In [ ]:
# Converts the mean and the standard deviation for a log-normal distribution over random variable X 
# to the mean and standard deviation for the normal distribution of EXP(X)
#
# Code based on: 
#   http://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal-data-with-specified-mean-and-variance.html
def convert_mu_lognormal_parameters(mu_lognormal, sigma_lognormal):
    phi = np.sqrt(sigma_lognormal + mu_lognormal**2)
    mu = np.log(mu_lognormal**2/phi)
    sigma = np.sqrt(np.log(phi**2/mu_lognormal**2))
    return mu, sigma

In [ ]:
# choose gift size metrics
giftSize_avg = 126
giftSize_std = giftSize_avg**2.2

# create log-normal distribution
mu_lognormal = giftSize_avg
sigma_lognormal = giftSize_std
mu, sigma = convert_mu_lognormal_parameters(mu_lognormal, sigma_lognormal)

# generate values
# Y: random variable of lognormal distribution
# np.exp(mu): mean of normal distribution over X, where X = log(Y)
# sigma: standard deviation of normal distribution over X, where X = log(Y)
giftSize_data = np.round(lognorm.rvs(s=sigma, scale=np.exp(mu), size=N) + 0.01, 
                         decimals=2)

## Billing State 
Modeling approximations, estimates, and assumptions:
- Billing state was modeled as a multinomial distribution, with the probability of drawing a particular state proportional to the state's population.


In [ ]:
# calculate relative state populations
total_pop = np.sum(df_pop_state['POPESTIMATE2016'])
df_pop_state['POPESTIMATE2016_RELATIVE'] = df_pop_state['POPESTIMATE2016'] / total_pop

In [ ]:
# generate values from multinomial distribution over states
pvals = df_pop_state['POPESTIMATE2016_RELATIVE']
billingState_data = []
for i in range(N):
    x = np.array([multinomial.rvs(1, pvals, size=1)])
    x = x.squeeze().astype(bool)
    billingState_data.append(df_pop_state['NAME'][x].iloc[0])

## Create CSV File

In [ ]:
file = open('../data/generated_raw_data.csv', 'w')
file.write("Age," + "Gender (male=1)," + "Gift Size," + "Billing State"  + "\n")
for i in range(N):
    file.write(str(age_data[i]) + ", " +
               str(gender_data[i]) + ", " +
               str(giftSize_data[i]) + ", " +
               billingState_data[i] + "\n")
file.close()

# Summarize the Data

In [ ]:
df = pd.read_csv("../data/" + "generated_raw_data.csv")

## Average Gift Size by State

In [ ]:
print(np.round(df.groupby(by='Billing State').mean()['Gift Size'], 2))

 ## Average Per Capita Gift Size 

In [ ]:
print(np.round(df['Gift Size'].mean(), 2))

## Average Gift Size by Gender

In [ ]:
print(np.round(df.groupby(by='Gender (male=1)').mean()['Gift Size'], 2))

## Histogram of Gift Size

In [ ]:
plt.figure(figsize=(15,5))

data = df['Gift Size']
plt.hist(data, bins=500)
plt.title("Gift Sizes of Sample Data (N=" + str(N) + ")")
plt.xlabel("Gift Size ($)")
plt.ylabel("Frequency")

plt.show()

print("Mean:   ", np.round(df['Gift Size'].mean(), 2))
print("Median: ", np.round(df['Gift Size'].median(), 2))
print("Min:    ", np.round(df['Gift Size'].min(), 2))
print("Max:    ", np.round(df['Gift Size'].max(), 2))

Unimodal:
- The majority of the density is centered on the far left side of the histogram. Most values are densely centered around the median of \$66.38.

Skewed Right:
- Moving past the median along the extended right tail of the histogram we see an exponential decay in gift sizes, with the maximum gift size reaching \$5,447.27.

Reasons:
- The histogram has this shape because most people tend to give gifts of a modest amount (\$1 - \$500), whereas there is an ever decreasing amount of people who are capable and willing of making gifts of larger and larger values.
- Another fact that pushes the histogram towards having a positive skew is that it is impossible to make a negative gift, but possible to give gifts up to an arbitrarily large amount.

## Histogram of Age

In [ ]:
plt.figure(figsize=(15,5))

max_age = 114
data = df['Age']
plt.hist(data, range=(0,max_age), bins=max_age+1)
plt.title("Ages of Sample Data (N=" + str(N) + ")")
plt.xlabel("Age (years)")
plt.ylabel("Frequency")

plt.show()

print("Mean:    ", np.round(df['Age'].mean(), 2))
print("Meadian: ", np.round(df['Age'].median(), 2))
print("Min:     ", np.round(df['Age'].min(), 2))
print("Max:     ", np.round(df['Age'].max(), 2))

Unform (0 to late 50's):
- The histogram is effectively uniform from the values of 0 to late 50's. 

Skewed Right:
- From the late 50's onward the histogram begins to decay exponentially stopping abruptly at a value of 113.

Reasons:
- The histogram is uniform from the values of 0 to late 50's because being born in any specific year is just as likely as any other. 
- The exponential decay in the histogram's right tail is caused by the onset of life threatening health conditions that become exponentially more likely every year once you pass your late 50's.

Caveats:
- The modeling procedures used did not take into account generational effects, which would make the likelihood of sampling people being born in certain years more likely than others.
- The modeling assumptions used lead to a generative model that consistently over samples very senior ages (i.e., 90+). This potentially could be avoided by modeling very senior ages with a separate exponential distribution then was used for ages 55 to 89. 

# Create Visualizations

In [ ]:
# Plots a selected measurment as heat-map across the 50 states of America.
#
# Code based on: 
#   https://plot.ly/python/choropleth-maps/
def visualize_state_meas_map(df, meas, title, unit=''):

    for col in df.columns:
        df[col] = df[col].astype(str)

    df['text'] = df['Billing State'] + '<br>' + meas + ": " + df[meas]

    data = [ dict(
            type='choropleth',
            autocolorscale = True,
            locations = df['State Code'],
            z = df[meas].astype(float),
            locationmode = 'USA-states',
            text = df['text'],
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                )
            ),
            colorbar = dict(
                title = unit
            )
        ) ]

    layout = dict(
            title = title,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)',
            ),
        )

    fig = dict(data=data, layout=layout)

    plotly.offline.iplot(fig, filename='d3-cloropleth-map')

## Average Gift Size by State

In [ ]:
# create DataFrame of average gift size by state
df_state_giftSize = \
    df.groupby(by='Billing State', as_index=False).mean()[['Billing State', 'Gift Size']]
df_state_giftSize['Gift Size'] = np.round(df_state_giftSize['Gift Size'], 2)

# add state codes to DataFrame (needed for plotly visualizations)
df_stateCodes = pd.read_csv('../data/state_codes.csv', header=None)
df_state_giftSize['State Code'] = df_stateCodes

# visualize
visualize_state_meas_map(df_state_giftSize.copy(), 'Gift Size', 'Average Gift Size by State', 
                         'USD')

## Number of Donations per State

In [ ]:
# create DataFrame of number of donations by state
df_state_numDonations = \
    df.groupby(by='Billing State', as_index=False).count()[['Billing State', 'Age']]
df_state_numDonations = \
    df_state_numDonations.rename(columns = {'Age': 'Number of Donations'})

# add state codes to DataFrame (needed for plotly visualizations)
df_stateCodes = pd.read_csv('../data/state_codes.csv', header=None)
df_state_numDonations['State Code'] = df_stateCodes

# visualize
visualize_state_meas_map(df_state_numDonations.copy(), 'Number of Donations', 
                         'Number of Donations by State', 'Donations')

# Build a Model

In [ ]:
df_lm = pd.DataFrame(data={
    'Population': df_pop_state['POPESTIMATE2016'].values,
    'Number of Donations': df_state_numDonations['Number of Donations'].values})

In [ ]:
# train linear model
lm = LinearRegression()
lm.fit(df_lm['Population'].values.reshape(-1, 1), 
       df_lm['Number of Donations'].values.reshape(-1, 1))
b = lm.intercept_[0]
m = lm.coef_[0][0]

# calculate variance explained
y_true = df_lm['Number of Donations'].values.reshape(-1, 1)
y_pred = b + (m * df_lm['Population'].values)
r2 = r2_score(y_true, y_pred)

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(x="Population", y="Number of Donations", data=df_lm, 
            line_kws={'color':"g", "alpha":0.5, "lw":4})
plt.title("Number of Donations by State Population")
plt.xlabel("State Population")
plt.show()

print("Percentage of Variance Explained (R-squared * 100) = ", round(r2 * 100, 2))

# General Caveats

- All generated variables in the analysis (i.e., age, gender, gift size, and billing state) were assumed to be independent from each other, which is clearly not the case. For example, knowing someone's age, gender, and billing state definitely gives you information useful in predicting/generating their gift size amount. 